# Set up

In [1]:
# update path with data dir
import sys
sys.path.append('../data/')
sys.path.append('../modelling/')

In [2]:
import player_data as player
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import patsy
from datetime import datetime
import numpy as np

/home/tom/footbot/venv/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
response_variable = 'goals_scored'

# Getting data

In [4]:
file_name = 'csvs/element_gameweek_' + datetime.today().strftime('%Y-%m-%d') + '.csv'

In [5]:
try:
    element_gameweek_df = pd.read_csv(file_name)
except:
    element_gameweek_df = player.get_element_gameweek_df()
    element_gameweek_df.to_csv(file_name)

In [6]:
current_event = 32
element_types = [2, 3, 4]
threshold_minutes = 1

In [7]:
element_gameweek_df =\
element_gameweek_df[
    (element_gameweek_df['event'] <= current_event)
    & (element_gameweek_df['element_type'].isin(element_types))
    & (element_gameweek_df['minutes'] >= threshold_minutes)
]

In [8]:
player.add_home_categorical_variable(element_gameweek_df)

In [9]:
element_gameweek_df['row_id'] = element_gameweek_df['row_id'].rank()

In [10]:
element_gameweek_df['element_fixture_rank'] = element_gameweek_df.groupby('element')['row_id'].rank()

In [11]:
rolling_cols = [
    'big_chances_missed',
    'goals_scored',
    'target_missed'
]

In [12]:
rolling_df = \
element_gameweek_df.copy().groupby('element', as_index=True)[['minutes'] + rolling_cols]\
.rolling(38, min_periods=1).sum().reset_index()[['element', 'minutes'] + rolling_cols]

In [13]:
rolling_df['element_fixture_rank'] = rolling_df.groupby('element')['minutes'].rank()

In [14]:
rolling_df['element_fixture_rank'] = rolling_df['element_fixture_rank'] + 1

In [15]:
element_gameweek_df =\
element_gameweek_df.join(
    rolling_df.set_index(['element', 'element_fixture_rank']),
    on=['element', 'element_fixture_rank'],
    rsuffix='_rolling')

In [16]:
for i in rolling_cols:
    element_gameweek_df[i + '_per_minute_rolling'] = \
    element_gameweek_df[i + '_rolling'] / element_gameweek_df['minutes_rolling']

In [17]:
rolling_p5_df = \
element_gameweek_df.copy().groupby('element', as_index=True)[['minutes'] + rolling_cols]\
.rolling(5, min_periods=1).sum().reset_index()[['element', 'minutes'] + rolling_cols]

In [18]:
rolling_p5_df['element_fixture_rank'] = rolling_p5_df.groupby('element')['minutes'].rank()

In [19]:
rolling_p5_df['element_fixture_rank'] = rolling_p5_df['element_fixture_rank'] + 1

In [20]:
element_gameweek_df =\
element_gameweek_df.join(
    rolling_p5_df.set_index(['element', 'element_fixture_rank']),
    on=['element', 'element_fixture_rank'],
    rsuffix='_rolling_p5')

In [21]:
for i in rolling_cols:
    element_gameweek_df[i + '_per_minute_rolling_p5'] = \
    element_gameweek_df[i + '_rolling_p5'] / element_gameweek_df['minutes_rolling']

In [22]:
element_gameweek_df.head()

,row_id,assists,attempted_passes,big_chances_created,big_chances_missed,bonus,bps,clean_sheets,clearances_blocks_interceptions,completed_passes,...,big_chances_missed_per_minute_rolling,goals_scored_per_minute_rolling,target_missed_per_minute_rolling,minutes_rolling_p5,big_chances_missed_rolling_p5,goals_scored_rolling_p5,target_missed_rolling_p5,big_chances_missed_per_minute_rolling_p5,goals_scored_per_minute_rolling_p5,target_missed_per_minute_rolling_p5
80,1.0,0,92,0,0,0,11,0,6,85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,2.0,0,74,0,0,0,17,0,4,70,...,0.0,0.0,0.011111,90.0,0.0,0.0,1.0,0.0,0.0,0.011111
83,3.0,0,38,0,0,0,10,0,2,36,...,0.0,0.0,0.006289,159.0,0.0,0.0,1.0,0.0,0.0,0.006289
84,4.0,0,64,0,0,0,19,0,10,57,...,0.0,0.0,0.004902,204.0,0.0,0.0,1.0,0.0,0.0,0.004902
85,5.0,0,89,0,0,0,17,0,2,87,...,0.0,0.0,0.003401,294.0,0.0,0.0,1.0,0.0,0.0,0.003401


In [23]:
element_gameweek_df.columns

Index(['row_id', 'assists', 'attempted_passes', 'big_chances_created',
       'big_chances_missed', 'bonus', 'bps', 'clean_sheets',
       'clearances_blocks_interceptions', 'completed_passes', 'creativity',
       'dribbles', 'ea_index', 'element', 'errors_leading_to_goal',
       'errors_leading_to_goal_attempt', 'fixture', 'fouls', 'goals_conceded',
       'goals_scored', 'ict_index', 'id', 'influence', 'key_passes',
       'kickoff_time', 'kickoff_time_formatted', 'loaned_in', 'loaned_out',
       'minutes', 'offside', 'open_play_crosses', 'opponent_team', 'own_goals',
       'penalties_conceded', 'penalties_missed', 'penalties_saved',
       'recoveries', 'red_cards', 'round', 'saves', 'selected', 'tackled',
       'tackles', 'target_missed', 'team_a_score', 'team_h_score', 'threat',
       'total_points', 'transfers_balance', 'transfers_in', 'transfers_out',
       'value', 'was_home', 'winning_goals', 'yellow_cards', 'event', 'team_a',
       'team_a_difficulty', 'team_h', 'team

# Neural network

In [24]:
def add_missing_columns(df, columns):
    for col in set(columns) - set(df.columns):
        df[col] = np.zeros(len(df))
    
    return df[columns]

In [25]:
rolling_vars = ' + '.join([i + '_per_minute_rolling' for i in rolling_cols])
rolling_p5_vars = ' + '.join([i + '_per_minute_rolling_p5' for i in rolling_cols])

formula = \
f'{response_variable} ~ C(element) + C(opposition_team) + was_home  + ' + rolling_vars

print(formula)

goals_scored ~ C(element) + C(opposition_team) + was_home  + big_chances_missed_per_minute_rolling + goals_scored_per_minute_rolling + target_missed_per_minute_rolling


In [26]:
y, X = patsy.dmatrices(formula, element_gameweek_df, return_type='dataframe')

In [27]:
len(X.columns)

482

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [29]:
X_train = add_missing_columns(X_train, X.columns)
X_test = add_missing_columns(X_test, X.columns)

In [30]:
# standardise explanatory data by subtracting the mean and scaling to unit variance
# find standardising transformations based on training set
scaler = StandardScaler().fit(X_train)

# standardise training set
X_train = scaler.transform(X_train)

# standardise test set
X_test = scaler.transform(X_test)

In [35]:
# instantiate model
model = Sequential()

# add the input layer, i.e. first hidden layer
model.add(Dense(units=25, activation='relu', input_shape=(len(X.columns),)))

# add a hidden layer
model.add(Dense(units=25, activation='relu'))

# add a hidden layer
model.add(Dense(units=25, activation='relu'))

# add a hidden layer
model.add(Dense(units=25, activation='relu'))
          
# add the output layer
model.add(Dense(units=1, activation='linear'))

In [36]:
# set training configuration
model.compile(
    loss='mean_absolute_error',
    optimizer='adam'
)

In [37]:
# train model on training set
model.fit(
    X_train, # explanatory variable training data
    y_train, # response variable training data
    epochs=20, # number of training iterations
    batch_size=10, # investigate this - The batch size that you specify in the code above defines the number of samples that going to be propagated through the network
    verbose=1 # monitor training progress
)

Epoch 1/20
6861/6861 [==============================] - 2s 225us/sample - loss: 0.1233
Epoch 2/20
6861/6861 [==============================] - 1s 170us/sample - loss: 0.1088
Epoch 3/20
6861/6861 [==============================] - 1s 162us/sample - loss: 0.1080
Epoch 4/20
6861/6861 [==============================] - 1s 166us/sample - loss: 0.1076
Epoch 5/20
6861/6861 [==============================] - 1s 167us/sample - loss: 0.1074
Epoch 6/20
6861/6861 [==============================] - 1s 168us/sample - loss: 0.1073
Epoch 7/20
6861/6861 [==============================] - 1s 164us/sample - loss: 0.1072
Epoch 8/20
6861/6861 [==============================] - 1s 165us/sample - loss: 0.1071
Epoch 9/20
6861/6861 [==============================] - 1s 161us/sample - loss: 0.1072
Epoch 10/20
6861/6861 [==============================] - 2s 246us/sample - loss: 0.1070
Epoch 11/20
6861/6861 [==============================] - 1s 197us/sample - loss: 0.1069
Epoch 12/20
6861/6861 [==================

In [38]:
# get predictions for test set
y_pred = model.predict(X_test).flatten()

# calculate cross entropy loss for test set
test_loss = (mean_absolute_error(y_test, y_pred), mean_squared_error(y_test, y_pred))
test_loss

(0.10714809532986061, 0.1263679500385161)